In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 88.7 kB in 2s (38.8 kB/s)
Reading package l

In [ ]:
# set options to be headless, ..
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:

def get_yt_live_urls(yt_channel_url):
  wd = webdriver.Chrome(options=options)
  wd.get(yt_channel_url)
  timeout = 5
  try:
    element_present = EC.presence_of_element_located((By.TAG_NAME, 'ytd-thumbnail-overlay-time-status-renderer'))
    WebDriverWait(wd, timeout).until(element_present)
  except TimeoutException:
    print("Timed out waiting for page to load")
    wd.quit()
  finally:
    print("Page loaded")

  yt_live_urls=[]
  yt_pages = [page for page in wd.find_elements_by_tag_name('ytd-browse') if page.get_attribute('page-subtype')=='channels']
  if yt_pages is None or len(yt_pages)==0:
    raise Exception('yt_pages not found')

  # print(yt_pages[0].get_attribute('outerHTML'))
  yt_page = yt_pages[0]
  yt_thumbnails = yt_page.find_elements_by_tag_name('ytd-thumbnail')
  if yt_thumbnails is None or len(yt_thumbnails) == 0:
    raise Exception('yt_thumbnails not found.')
  
  for r in yt_thumbnails:
    # check if time is 'LIVE' in the thumbnails
    yt_time = r.find_element_by_tag_name('ytd-thumbnail-overlay-time-status-renderer')
    if yt_time is None:
      raise Exception('yt_time cannot be None')
    if yt_time.get_attribute('innerText').strip('\n').strip(' ')=='LIVE':
      yt_link = r.find_element_by_tag_name('a')
      yt_link_href = yt_link.get_attribute('href')
      yt_live_urls.append(yt_link_href)
    
  wd.quit()
  return yt_live_urls

In [ ]:
get_yt_live_urls('https://www.youtube.com/channel/UCczq0lf4nd5GHG-2Pjoz4Vw')

Page loaded


['https://www.youtube.com/watch?v=qumYPTrINUM']